In [1]:
import os
from glob import glob
import pickle

import nibabel as nib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from   skimage.measure import find_contours
from   scipy.ndimage import gaussian_filter

import pydicom
from   pydicom.dataset import FileDataset, Dataset
from   pydicom.uid import ExplicitVRLittleEndian
import pydicom._storage_sopclass_uids

In [2]:
# Utils function to load and save nifti files with the nibabel package
def load_nii(img_path):
    nimg = nib.load(img_path)
    return nimg.get_data(), nimg.affine, nimg.header

def save_nii(img_path, data, affine, header):
    nimg = nib.Nifti1Image(data, affine=affine, header=header)
    nimg.to_filename(img_path)

In [3]:
basepath = 'original_data'
patients = [f for f in os.listdir(basepath)]
folders  = [os.path.join(basepath, f) for f in patients]
folders  = [f for f in folders if os.path.isdir(f)]

In [4]:
print(folders)

['original_data/patient001', 'original_data/patient003', 'original_data/patient002']


In [5]:
def get_img_conts_paths(folder):
    subfolders = os.listdir(folder)
    img_path   = [f for f in subfolders if '_4d' in f][0]
    img_path   = os.path.join(folder, img_path)
    imgs, _, header = load_nii(img_path)
    segs = [f for f in subfolders if '_gt' in f]
    segs1,_,_ = load_nii(os.path.join(folder, segs[0]))
    phase1    = int(segs[0].split('frame')[1].split('_')[0])
    segs2,_,_ = load_nii(os.path.join(folder, segs[1]))
    phase2    = int(segs[1].split('frame')[1].split('_')[0])
    return imgs, header, segs1, phase1, segs2, phase2

imgs, header, segs1, phase1, segs2, phase2 = get_img_conts_paths(folders[0])
print(imgs.shape)
print(segs1.shape)
print(phase1)
print(segs2.shape)
print(phase2)
print(header)
#plt.imshow(segs1[:, :, 0])

<ipython-input-2-4cc24b554201>:4: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  return nimg.get_data(), nimg.affine, nimg.header


(216, 256, 10, 30)
(216, 256, 10)
12
(216, 256, 10)
1
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 16384
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  4 216 256  10  30   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [ 1.      1.5625  1.5625 10.      1.      1.      1.      1.    ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 614.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 1071
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : unknown
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : 0

In [6]:
def store_anno(segm, sop):
    pass

def store_img(img, header, sl_n, ph_n, img_nr, case):
    img = img.astype(np.uint16)
    meta = pydicom.Dataset()
    meta.MediaStorageSOPClassUID = pydicom._storage_sopclass_uids.MRImageStorage
    meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
    meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian  
    ds = Dataset()
    ds.file_meta = meta
    ds.SOPInstanceUID = pydicom.uid.generate_uid()
    ds.is_little_endian = True
    ds.is_implicit_VR = False
    ds.SOPClassUID = pydicom._storage_sopclass_uids.MRImageStorage
    ds.PatientName = case
    ds.PatientID = "123456"
    ds.Modality = "MR"
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.FrameOfReferenceUID = pydicom.uid.generate_uid()
    ds.SeriesDescription = "cine ssfp sax multislice"
    ds.SliceLocation = sl_n
    ds.SliceThickness = str(header['pixdim'][3])
    ds.SpacingBetweenSlices = str(header['pixdim'][3])
    res = header['pixdim'][1]
    ds.PixelSpacing = str(res)+'\\'+str(res)
    ds.SeriesNumber = ph_n
    ds.BitsStored = 16
    ds.BitsAllocated = 16
    ds.SamplesPerPixel = 1
    ds.HighBit = 15
    ds.ImagesInAcquisition = "1"
    ds.Rows = img.shape[0]
    ds.Columns = img.shape[1]
    ds.InstanceNumber = ph_n
    ds.ImagePositionPatient = r"0\0\1"
    ds.ImageOrientationPatient = r"1\0\0\0\-1\0"
    ds.ImageType = r"ORIGINAL\PRIMARY\AXIAL"
    ds.RescaleIntercept = "0"
    ds.RescaleSlope     = "1"
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.PixelRepresentation = 1
    pydicom.dataset.validate_file_meta(ds.file_meta, enforce_standard=True)
    ds.PixelData = img.tobytes()
    filename = os.path.join('Imgs', case, 'img_'+str(img_nr)+'.dcm')
    ds.save_as(filename=filename, write_like_original=False)
    return ds.SOPInstanceUID


def store_contour(sop, res, mask, case, i, smooth=False):
    h, w = mask.shape[0], mask.shape[1]
    mask1 = gaussian_filter(mask==1, sigma=0.2) if smooth else mask==1
    mask2 = gaussian_filter(np.logical_or(mask==2, mask==3), sigma=0.2) if smooth else np.logical_or(mask==2, mask==3)
    mask3 = gaussian_filter(mask==3, sigma=0.2) if smooth else mask==3
    conts_rv   = [c.tolist() for c in find_contours(mask1, 0.5)]
    conts_epi  = [c.tolist() for c in find_contours(mask2, 0.5)]
    conts_endo = [c.tolist() for c in find_contours(mask3, 0.5)]
    if len(conts_rv)  !=0: conts_rv  =[[[b,a] for c in conts_rv   for [a,b] in c]]
    if len(conts_endo)!=0: conts_endo=[[[b,a] for c in conts_endo for [a,b] in c]]
    if len(conts_epi) !=0: conts_epi =[[[b,a] for c in conts_epi  for [a,b] in c]]
    d = {'lv endocardium':      {'contour': conts_endo, 
                                 'type': ['type', '  FREE'], 
                                 'subpixel resolution': 1, 
                                 'width':  w, 
                                 'height': h, 
                                 'pixel size': [res, res]}, 
         'lv epicardium':       {'contour': conts_epi, 
                                 'type': ['type', '  FREE'], 
                                 'subpixel resolution': 1, 
                                 'width':  w, 
                                 'height': h, 
                                 'pixel size': [res, res]}, 
         'lv papillary muscle': {'contour': []}, 
         'rv endocardium':      {'contour': conts_rv, 
                                 'type': ['type', '  FREE'], 
                                 'subpixel resolution': 1, 
                                 'width':  w, 
                                 'height': h, 
                                 'pixel size': [res, res]},  
         'rv epicardium': {'contour': []}, 
         'rv papillary muscle': {'contour': []}, 
         None: {'contour': []}, 
         'left atrium': {'contour': []}, 
         'viewer': {'hotspot': [93.8, 102.1], 'zoom':  4.0},   # ?
         'window': {'center':  372.0,         'width': 890.0}} # ?
    segm = 'Smooth' if smooth else 'Gold'
    filepath = os.path.join(segm, case, 'sub_annotations', sop+'.pickle')
    pickle.dump(d, open(filepath, 'wb'))

In [7]:
def create_folder(case):
    if not os.path.exists('Imgs'):   os.mkdir('Imgs')
    if not os.path.exists('Gold'):   os.mkdir('Gold')
    if not os.path.exists('Smooth'): os.mkdir('Smooth')
    if not os.path.exists(os.path.join('Imgs',   case)): os.mkdir(os.path.join('Imgs',   case))
    if not os.path.exists(os.path.join('Gold',   case)): os.mkdir(os.path.join('Gold',   case))
    if not os.path.exists(os.path.join('Smooth', case)): os.mkdir(os.path.join('Smooth', case))
    if not os.path.exists(os.path.join('Gold', case, 'sub_annotations')):
        os.mkdir(os.path.join('Gold', case, 'sub_annotations'))
    if not os.path.exists(os.path.join('Smooth', case, 'sub_annotations')):
        os.mkdir(os.path.join('Smooth', case, 'sub_annotations'))
    #contour_file = open(os.path.join('Gold',   case, 'contours.txt'),"w")
    #contour_file.close()
    return

In [8]:
for i in range(len(folders)):
    f = folders[i]
    p = patients[i]
    create_folder(p)
    print(i)
    imgs, header, segs1, phase1, segs2, phase2 = get_img_conts_paths(f)
    nr_slices, nr_phases = imgs.shape[2], imgs.shape[3]
    for sl_n in range(nr_slices):
        for ph_n in range(nr_phases):
            img_nr = sl_n * nr_phases + ph_n
            img = imgs[:,:,sl_n,ph_n]
            sop = store_img(img, header, sl_n, ph_n, img_nr, p)
            res = header['pixdim'][1]
            if ph_n == phase1:
                mask = segs1[:,:,sl_n]
                store_contour(sop, res, mask, p, img_nr)
                store_contour(sop, res, mask, p, img_nr, True)
            elif ph_n == phase2:
                mask = segs2[:,:,sl_n]
                store_contour(sop, res, mask, p, img_nr)
                store_contour(sop, res, mask, p, img_nr, True)
            else: 
                mask = np.zeros_like(img)
                store_contour(sop, res, mask, p, img_nr)
                store_contour(sop, res, mask, p, img_nr, True)

0


<ipython-input-2-4cc24b554201>:4: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  return nimg.get_data(), nimg.affine, nimg.header


1
2
